In [1]:
import numpy as np
import pandas as pd
import pylab
from obj_function import energy_objective, uccsd_singlet_paramsize

import sys
#May need to link to scikit-quant/python location
from skquant.opt.imfil import imfil, optset


/home/ana-tudor/quantum/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ana-tudor/quantum/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'obj_function'

In [2]:
def generate_input_amps(num):
    original_electrons = 16
    n_electrons = 2
    n_orbitals = 2

    occupied_indices = range(8 - n_electrons // 2)
    active_indices = range(8 - n_electrons // 2, 8 - n_electrons // 2 + n_orbitals)
    active_electrons = original_electrons - len(occupied_indices) * 2
    active_qubits = len(active_indices) * 2
    
    results = []
        
    for i in range(num):
        n_amplitudes = int(uccsd_singlet_paramsize(active_qubits, active_electrons))
        current_amplitudes = [0.0] * n_amplitudes + .002 * np.random.randn(n_amplitudes)
        current_amplitudes = current_amplitudes / np.linalg.norm(current_amplitudes)
        results.append(current_amplitudes)
        
    return results
        

NameError: name 'uccsd_singlet_paramsize' is not defined

In [ ]:
init_params = generate_input_params(20)
noises = [0.0, 0.00005, .002]
bounds = numpy.matrix([[-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.], 
                   [-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.],
                   [-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.],
                   [-1., 1.], [-1., 1.], [-1., 1.], [-1., 1.]])
budget = 80

results = {}
for noise in noises:
    results['{}_init_params'.format(noise)] = []
    results['{}_fin_params'.format(noise)] = []
    results['{}_iters'.format(noise)] = []
    results['{}_fin_energy'.format(noise)] = []

optout = optset()
optout = optset(optout, random_stencil=2)
optout = optset(optout, fscale=-1.1)

print(optout)


In [ ]:
for noise in noises:
    #Setting noise for ethylene notebooks will be different
    
    for x in init_params:
        
        print("RUNNING PARAMETER")
        for ext_iter in range(3):
            print("RUNNING ITERATION ", ext_iter)
            optout = optset(optout, svarmin=noise)
            
            x, histout= imfil(x0, energy_objective, budget, bounds, optout)
            results['{}_init_param'.format(noise)].append(param)
            results['{}_fin_param'.format(noise)].append(x)
            results['{}_iters'.format(noise)].append(ext_iter)
#             print(x)
#             print(histout)
            
            
        
print(results)

print("FINISHED")

In [ ]:
def optimal_value(y):
    return min(y)

def absolute_error(fvalue):
    global target
    return numpy.abs(fvalue - target)

    
def gen_plot(x,y):
    fig = pylab.figure(figsize=(6,3))
    pylab.plot(x,y, 'ro')

    
def errors(y_list):
    return [absolute_error(y) for y in y_list]


def relative_errors(y_list):
    minim = optimal_value(y_list)
    print(y_list)
    print(minim)
    return [numpy.abs(minim-y) for y in y_list]
    
    
def gen_error_plot(iters, fvals):
    gen_plot(iters,relative_errors(fvals))
    pylab.ylabel('Error')
    pylab.xlabel('External iteration')
    pylab.title('Error varying by external iterations')

    
def gen_fvalue_vs_iters(iters,fval):
    gen_plot(iters, fval)
    pylab.xlabel('Iteration')
    pylab.ylabel('Function values')
    pylab.title('Function values varying by number of iterations')
    
def gen_fvalue_vs_params(cols, y_list):
    for i in range(len(cols)):
        fig = pylab.figure(figsize=(4,3))
        pylab.plot(cols[i], y_list, 'ro')
        pylab.xlabel('Parameter ' + str(i))
        pylab.ylabel('Function values')
        pylab.title('Function values versus param 1')
    
           
def cal_fin_energies(param_list):
    return []

def display_data(res, n):
    global noise
    noise = n
    iters = res['{}_iters'.format(noise)]
    fin_param = res['{}_fin_param'.format(noise)]
    params = [[x[i].item(0,0) for x in fin_param] for i in range(len(fin_param))]
    param1 = [x[0].item(0,0) for x in fin_param]
    fin_energies = [energy_objective(param) for param in fin_param]
    
    gen_fvalue_vs_params(params, fin_energies)
    
    gen_fvalue_vs_iters(iters, fin_energies)
    
    gen_error_plot(iters, fin_energies)
    
